In [30]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam
import numpy as np

torch.manual_seed(1)

In [31]:
# two dice one is fair, one is loaded
# emission matrix i.e. p(observing a 0|fair dice)
fair_dice = np.array([1/6]*6)
loaded_dice = np.array([0.04,0.04,0.04,0.04,0.04,0.8])

probabilities = {'fair': fair_dice,
                'loaded': loaded_dice}
# print(fair_dice.reshape(-1,1))
# print(loaded_dice.reshape(-1,1))
# h1 = fair_dice.reshape(-1,1)
# h2 = loaded_dice.reshape(-1,1)
# print(np.hstack([h1, h2]))
log_likelihood = np.hstack([np.log(fair_dice).reshape(-1,1),
                            np.log(loaded_dice).reshape(-1,1)])

# 1st col: log(observing 1/2/3/4/5/6 of a fair dice); 2nd col: log(observing 1/2/3/4/5/6 of a loaded dice)
print(log_likelihood)


# transition matrix between states (start, fair dice, loaded dice)
# if dice is fair at time t, 0.6 chance we stay fair, 0.4 chance it is loaded at time 2
transition_mat = {'fair': np.array([0.6, 0.4, 0.0]),
                 'loaded': np.array([0.3, 0.7, 0.0]),
                 'start': np.array([0.5, 0.5, 0.0])}
states = ['fair', 'loaded', 'start']
state2ix = {'fair': 0,
           'loaded': 1,
           'start': 2}

[[-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -3.21887582]
 [-1.79175947 -0.22314355]]


In [32]:
# initialize transition matrix
# p(i, j) = p(current state is i | prev state is j)
n_dice = 2
n_states = n_dice
transition = torch.nn.init.normal(nn.Parameter(torch.randn(n_dice, n_dice + 1)), -1, 0.1)
print(transition)

Parameter containing:
-1.1523 -0.9618 -1.1028
-1.0563 -1.0892 -1.0058
[torch.FloatTensor of size 2x3]



In [85]:
class CRF(nn.Module):
    
    def __init__(self, n_dice, log_likelihood):
        super(CRF, self).__init__()
        
        self.n_states = n_dice
        self.transition = torch.nn.init.normal(nn.Parameter(torch.randn(n_dice, n_dice + 1)), -1, 0.1)
        self.loglikelihood = log_likelihood
    

    def to_scalar(self, var):
#         print("to list: ", var.view(-1).data.tolist())
        return var.view(-1).data.tolist()[0]


    def argmax(self, vec):
        # return the index of the max value for each row
        _, idx = torch.max(vec, 1)
        return self.to_scalar(idx)
        
    # Compute log sum exp in a numerically stable way for the forward algorithm
    # Source: http://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html
    def log_sum_exp(self, vec):
#         print("self.argmax(vec): ", self.argmax(vec))
        max_score = vec[0, self.argmax(vec)]
#         print("max_score: ", max_score)
#         print("max_score.view(1, -1): ", max_score.view(1, -1))
        max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
#         print("vec.size(): ", vec.size())
#         print("max_score_broadcast: ", max_score_broadcast)
#         print("vec - max_score_broadcast: ", vec - max_score_broadcast)
        return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    
    
    def _data_to_likelihood(self, rolls):
        """Converts a numpy array of rolls (integers) to log-likelihood.
        Input is one [1, n_rolls]
        """
        return Variable(torch.FloatTensor(self.loglikelihood[rolls]), requires_grad=False)
    
    def _compute_likelihood_numerator(self, loglikelihoods, states):
        # Let y be a tag sequence and x an input sequence of words
        # p(y|x) = exp(Score(x, y))/sum_over_all_possible_sequences_(ie y')(exp(Score(x, y')))
        # here returns Score(x, y) for a observed sequence of y
        
        """Computes numerator of likelihood function for a given sequence.
        
        We'll iterate over the sequence of states and compute the sum 
        of the relevant transition cost with the log likelihood of the observed
        roll. 
        Input:
            loglikelihoods: torch Variable. Matrix of n_obs x n_states. 
                            i,j entry is loglikelihood of observing roll i given state j
            states: sequence of labels 
        Output:
            score: torch Variable. Score of assignment. 
        """
    
        prev_state = self.n_states # initialized: n_states = n_dices 
        score = Variable(torch.Tensor([0]))
        for index, state in enumerate(states): # a sequence of labels (i.e. [fair, fair, loaded, fair, ...])
            # recall transition is parameter, shape [n_states, n_states+1], normal(-1, 0.1)
            # loglikelihoods(emission matrix) passed by the argument
            score += self.transition[state, prev_state] + loglikelihoods[index, state]
            prev_state = state
        return score
    
        
    def _compute_likelihood_numerator(self, loglikelihoods, states):
        """Computes numerator of likelihood function for a given sequence.
        
        We'll iterate over the sequence of states and compute the sum 
        of the relevant transition cost with the log likelihood of the observed
        roll. 
        Input:
            loglikelihoods: torch Variable. Matrix of n_obs x n_states. 
                            i,j entry is loglikelihood of observing roll i given state j
            states: sequence of labels
        Output:
            score: torch Variable. Score of assignment. 
        """
        prev_state = self.n_states
        score = Variable(torch.Tensor([0]))
        for index, state in enumerate(states):
            score += self.transition[state, prev_state] + loglikelihoods[index, state]
            prev_state = state
        return score
    
    
    def _compute_likelihood_denominator(self, loglikelihoods):
        """Implements the forward pass of the forward-backward algorithm.
        
        We loop over all possible states efficiently using the recursive
        relationship: alpha_t(j) = \sum_i alpha_{t-1}(i) * L(x_t | y_t) * C(y_t | y{t-1} = i)
        Input:
            loglikelihoods: torch Variable. Same input as _compute_likelihood_numerator.
                            This algorithm efficiently loops over all possible state sequences
                            so no other imput is needed.
        Output:
            torch Variable. 
        """
        # loglikelihood is like the emitt matrix w.r.t.observed values (a sentence):
        # matrix of len(sentence) * tagset_size
        
        print("self.transition:\n", self.transition)
        print("self.transition[:, self.n_states]:\n", self.transition[:, self.n_states])
        print("loglikelihoods:\n", loglikelihoods)
        print("loglikelihoods[0].view(1, -1):\n", loglikelihoods[0].view(1, -1))
        
        # Stores the current value of alpha at timestep t
        prev_alpha = self.transition[:, self.n_states] + loglikelihoods[0].view(1, -1)
        print("prev_alpha:\n", prev_alpha)
        print("start loop:\n")
        for roll in loglikelihoods[1:]:
            print("current roll: ", roll)
            alpha_t = []

            # Loop over all possible states
            for next_state in range(self.n_states):
                print("current next_state: ", next_state)
                print("self.transition[next_state,:self.n_states].view(1, -1):\n", 
                      self.transition[next_state,:self.n_states].view(1, -1))
                print("roll[next_state].view(1, -1):\n", 
                      roll[next_state].view(1, -1))
                print("roll[next_state].view(1, -1).expand(1, self.n_states):\n", 
                      roll[next_state].view(1, -1).expand(1, self.n_states))
                # Compute all possible costs of transitioning to next_state
                feature_function = self.transition[next_state,:self.n_states].view(1, -1) +\
                                   roll[next_state].view(1, -1).expand(1, self.n_states)
                
                print("feature_function: ", feature_function)
                alpha_t_next_state = prev_alpha + feature_function
                print("alpha_t_next_state:\n", alpha_t_next_state)
                alpha_t.append(self.log_sum_exp(alpha_t_next_state))
                print("log_sum_exp(alpha_t_next_state): ", self.log_sum_exp(alpha_t_next_state))
                
            prev_alpha = torch.cat(alpha_t).view(1, -1)
            print("for current roll: prev_alpha\n", prev_alpha)
        return self.log_sum_exp(prev_alpha)
    
    
    def _viterbi_algorithm(self, loglikelihoods):
        """Implements Viterbi algorithm for finding most likely sequence of labels.
        
        Very similar to _compute_likelihood_denominator but now we take the maximum
        over the previous states as opposed to the sum. 
        Input:
            loglikelihoods: torch Variable. Same input as _compute_likelihood_denominator.
        Output:
            tuple. First entry is the most likely sequence of labels. Second is
                   the loglikelihood of this sequence. 
        """
        
        argmaxes = []

        # prev_delta will store the current score of the sequence for each state
        # prev_delta for the first observation
        prev_delta = self.transition[:, self.n_states].contiguous().view(1, -1) +\
                      loglikelihoods[0].view(1, -1)

        print("self.transition:\n", self.transition)
        print("self.transition[:, self.n_states]:\n", self.transition[:, self.n_states])
        print("loglikelihoods:\n", loglikelihoods)
        print("loglikelihoods[0].view(1, -1):\n", loglikelihoods[0].view(1, -1))
        print("prev_delta:\n", prev_delta)
        print("start loop:\n")
        
        for roll in loglikelihoods[1:]:
            print("current roll: ", roll)
            local_argmaxes = []
            next_delta = []
            for next_state in range(self.n_states):
                print("current next_state: ", next_state)
                # we don't add emission here as it doesn't effect argmax
                # since we look for most likely previous path that lead to this state, 
                # emssion score that relate this state to features at this timestep is irrelavent
                feature_function = self.transition[next_state,:self.n_states].view(1, -1) + prev_delta
                print("feature function:\n", feature_function)
                # to figure out the best bp state (most likely previous state that leads to this state)
                most_likely_state = self.argmax(feature_function)
                local_argmaxes.append(most_likely_state)
                score = feature_function[0][most_likely_state].view(1)
                next_delta.append(score)
                print("next_delta.append: ", score)
                print("local_argmaxes.append: ", most_likely_state)
                
            # add emission part here to viterbi score for each state
            prev_delta = (torch.cat(next_delta) + roll).view(1, -1)
            print("prev_delta for this roll:\n", prev_delta)
            argmaxes.append(local_argmaxes)
            print("argmaxes.append after this roll: ", local_argmaxes)
        
        final_state = self.argmax(prev_delta)
        final_score = prev_delta[0][final_state]
        print("final_state: ", final_state)
        print("final_score: ", final_score)
        
        path_list = [final_state]
        # Backtrack through the argmaxes to find most likely state
        for states in reversed(argmaxes):
            final_state = states[final_state]
            path_list.append(final_state)
        print("best path before reverse: ", path_list)
        path_list.reverse()
        return final_score, path_list

In [86]:
a = torch.autograd.Variable(torch.randn(2, 3))
print("== a ==\n", a)
crf = CRF(2, log_likelihood)
print("== crf.argmax ==\n", crf.argmax(a))
print("== crf.log_sum_exp == \n", crf.log_sum_exp(a))

== a ==
 Variable containing:
 0.4903  1.0318 -0.5989
 1.6015 -1.0735 -1.2173
[torch.FloatTensor of size 2x3]

== crf.argmax ==
 1
== crf.log_sum_exp == 
 Variable containing:
 2.3596
[torch.FloatTensor of size 1]



In [87]:
rolls = np.array([1, 3, 5, 0])
print(rolls.shape)
print(crf._data_to_likelihood(rolls))

(4,)
Variable containing:
-1.7918 -3.2189
-1.7918 -3.2189
-1.7918 -0.2231
-1.7918 -3.2189
[torch.FloatTensor of size 4x2]



In [88]:
states = [0,0,1,0]
numerator_score = crf._compute_likelihood_numerator(log_likelihood, states)
print(numerator_score)

Variable containing:
-12.5024
[torch.FloatTensor of size 1]



In [89]:
log_likelihoods = crf._data_to_likelihood(rolls)
crf._compute_likelihood_denominator(log_likelihoods)

self.transition:
 Parameter containing:
-1.0461 -1.0099 -0.9527
-0.8995 -1.0287 -1.1162
[torch.FloatTensor of size 2x3]

self.transition[:, self.n_states]:
 Variable containing:
-0.9527
-1.1162
[torch.FloatTensor of size 2]

loglikelihoods:
 Variable containing:
-1.7918 -3.2189
-1.7918 -3.2189
-1.7918 -0.2231
-1.7918 -3.2189
[torch.FloatTensor of size 4x2]

loglikelihoods[0].view(1, -1):
 Variable containing:
-1.7918 -3.2189
[torch.FloatTensor of size 1x2]

prev_alpha:
 Variable containing:
-2.7445 -4.3351
[torch.FloatTensor of size 1x2]

start loop:

current roll:  Variable containing:
-1.7918
-3.2189
[torch.FloatTensor of size 2]

current next_state:  0
self.transition[next_state,:self.n_states].view(1, -1):
 Variable containing:
-1.0461 -1.0099
[torch.FloatTensor of size 1x2]

roll[next_state].view(1, -1):
 Variable containing:
-1.7918
[torch.FloatTensor of size 1x1]

roll[next_state].view(1, -1).expand(1, self.n_states):
 Variable containing:
-1.7918 -1.7918
[torch.FloatTensor of s

Variable containing:
-8.7198
[torch.FloatTensor of size 1]

In [90]:
score, best_path = crf._viterbi_algorithm(log_likelihoods)
print("==final score: ", score)
print("==best path: ", best_path)

self.transition:
 Parameter containing:
-1.0461 -1.0099 -0.9527
-0.8995 -1.0287 -1.1162
[torch.FloatTensor of size 2x3]

self.transition[:, self.n_states]:
 Variable containing:
-0.9527
-1.1162
[torch.FloatTensor of size 2]

loglikelihoods:
 Variable containing:
-1.7918 -3.2189
-1.7918 -3.2189
-1.7918 -0.2231
-1.7918 -3.2189
[torch.FloatTensor of size 4x2]

loglikelihoods[0].view(1, -1):
 Variable containing:
-1.7918 -3.2189
[torch.FloatTensor of size 1x2]

prev_delta:
 Variable containing:
-2.7445 -4.3351
[torch.FloatTensor of size 1x2]

start loop:

current roll:  Variable containing:
-1.7918
-3.2189
[torch.FloatTensor of size 2]

current next_state:  0
feature function:
 Variable containing:
-3.7906 -5.3450
[torch.FloatTensor of size 1x2]

next_delta.append:  Variable containing:
-3.7906
[torch.FloatTensor of size 1]

local_argmaxes.append:  0
current next_state:  1
feature function:
 Variable containing:
-3.6440 -5.3638
[torch.FloatTensor of size 1x2]

next_delta.append:  Variable 